In [ ]:
import os
import requests
import uuid
import json

class GigaChat:
    def __init__(self, sb_auth_data):
        self.sb_auth_data = sb_auth_data
        self.auth_token = self._get_auth_token()

    def _get_auth_token(self):
        unique_id = str(uuid.uuid4())
        url = "https://ngw.devices.sberbank.ru:9443/api/v2/oauth"

        payload = 'scope=GIGACHAT_API_PERS'
        headers = {
            'Content-Type': 'application/x-www-form-urlencoded',
            'Accept': 'application/json',
            'RqUID': unique_id,
            'Authorization': f'Basic {self.sb_auth_data}'
        }

        # Запрос токена доступа
        response = requests.post(url, headers=headers, data=payload, verify=False)
        
        # Получение и возврат access_token из JSON-ответа
        if response.status_code == 200:
            access_token = json.loads(response.text)['access_token']
            return f"Bearer {access_token}"
        else:
            raise Exception(f"Error: Unable to obtain access token, status code {response.status_code}")

    def invoke_gigachat(self, messages, model="GigaChat", raw=True, system_prompt=None):
        if system_prompt:
            messages = [{"role": "system", "content": system_prompt}] + messages
            
        url = "https://gigachat.devices.sberbank.ru/api/v1/chat/completions"
        
        payload = json.dumps({
            "model": model,
            "messages": messages,
            "stream": False,
            "repetition_penalty": 1
        })

        headers = {
            'Content-Type': 'application/json',
            'Accept': 'application/json',
            'Authorization': self.auth_token
        }

        try:
            response = requests.post(url, headers=headers, data=payload, verify=False)
            
            # Проверка на код 200 (OK)
            if response.status_code == 200:
                if raw:
                    return response
                else:
                    msg = json.loads(response.text)['choices'][0]['message']
                    return msg
            else:
                # Обработка статусов, отличных от 200
                return f"Error: Server returned status code {response.status_code}"
        except requests.RequestException as e:
            # Обработка исключений при запросе
            return f"Request failed: {str(e)}"


In [ ]:
os.environ['SB_AUTH_DATA'] = 'token'
# Инициализация объекта GigaChat с аутентификационными данными
sb_auth_data = os.getenv('SB_AUTH_DATA')
gigachat = GigaChat(sb_auth_data)

# Подготовка сообщения
user_qustion = 'Вопрос, который пользователь задал ассистенту'
parsed_text = 'Текст, который мы спарсили с посещенных сайтов (желательно не слишком большой для экономии токенов)'

text = f"""
Ответь на вопрос пользователя, основываясь на полученном контексте.

Вопрос пользователя:
{user_qustion}

Контекст:
{parsed_text}
"""
user_msg = {
    'content': text,
    'role': 'user',
}

# Системный промпт
system_prompt = """
You are a Q&A assistant. Your goal is to answer questions as
accurately as possible based on the instructions and context provided.
"""

# Отправка запроса и вывод ответа
model_response = gigachat.invoke_gigachat(messages=[user_msg], system_prompt=system_prompt)
print(json.loads(model_response.text)['choices'][0]['message']['content'])